In [1]:
from pyspark.sql import SparkSession

## Starting SparkSession
spark = SparkSession.builder\
    .appName("HockeyAnalysis")\
    .master("local[*]")\
    .getOrCreate()

spark

In [2]:
from pyspark.sql.functions import col, row_number, rank, dense_rank, sum,monotonically_increasing_id

from pyspark.sql import Window


In [3]:
## Reading the CSV files 

## reading the raw date dimension table provided
date_csv_dataframe = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("./data/DIM.Date.Table.csv")

## reading the raw hockey player table provided
hockey_src1_dataframe = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("./data/Hockey_Src1.csv")

## reading the raw hockey team table provided
hockey_src2_dataframe = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("./data/Hockey_Src2.csv")

In [4]:
## 

extracted_league_table = hockey_src2_dataframe.select(col("League_Name"))\
    .distinct()\
    .filter(col("League_Name") != 'NA')\
    .orderBy(col("League_Name"))

window_spec = Window.orderBy("League_Name")

league_dimension_table = extracted_league_table\
    .withColumn("League_Id", rank().over(window_spec))\
    .select("League_Id", "League_Name") 

league_pandas_df = league_dimension_table.toPandas()
league_pandas_df.to_csv('tables/dimension/league_dimension.csv', index= False)

In [30]:
extracted_club_table = hockey_src2_dataframe\
    .select(col("League_Name"), col("Club_Name"), col("Manager"), col("Owner"))\
    .distinct()\
    .filter(col("Club_Name") != 'NA')\

extracted_club_table = extracted_club_table\
    .withColumn("Club_Id", rank().over(Window.orderBy(col("Club_Name"))))\
    .select(col("Club_Id"), col("League_Name"), col("Club_Name"), col("Manager"), col("Owner"))

club_dimension_table = extracted_club_table.join(league_dimension_table,
                                       on=extracted_club_table.League_Name == league_dimension_table.League_Name,
                                       how = 'outer')\
    .select(col("Club_Id"), col("League_Id"), col("Club_Name"), col("Manager"), col("Owner"))\
    .orderBy(col("Club_Id"))

print(club_dimension_table.show())

club_pandas_df = club_dimension_table.toPandas()
club_pandas_df.to_csv('tables/dimension/club_dimension.csv',index=False)

+-------+---------+------------------+----------------+----------------+
|Club_Id|League_Id|         Club_Name|         Manager|           Owner|
+-------+---------+------------------+----------------+----------------+
|      1|        3| Atheletico Madrid|     Alvin Lucas|      Dean Green|
|      2|        2|           Arsenal| Caesar Townsend|Brandon Townsend|
|      3|        2|       Aston Villa|  Hamish Clayton|Sylvester Campos|
|      4|        3| Atheletico Bilbao|   Murphy Strong|      Erich Bird|
|      5|        3|         Barcelona|   Malik Sanders|    Curran Drake|
|      6|        1| Bayern Leverkusan| Mollie Ferguson|     Hilel Nixon|
|      7|        1|     Bayern Munich|    Yvonne Bryan|   Erich Navarro|
|      8|        2|           Chelsea|      Bruce Bray|       Beck Bell|
|      9|        1|          Dortmund|      Ursa Finch|    Yasir Conley|
|     10|        2|           Everton|    Hayes Bowman|       Roth Rice|
|     11|        1|           FC Koln|  Dante Casti

In [29]:
extracted_country_table = hockey_src2_dataframe\
    .select(col("Country Name"), col("Coach"))\
    .distinct()\
    .filter(col("Country Name") != 'NA')\

country_dimension_table = extracted_country_table\
    .withColumn("Country_Id", rank().over(Window.orderBy(col("Country Name"))))\
    .withColumn("Country_Name" , col("Country Name"))\
    .select("Country_Id", "Country_Name", "Coach")

print(country_dimension_table.show())

country_pandas_df = country_dimension_table.toPandas()
country_pandas_df.to_csv('tables/dimension/country_dimension.csv', index = False)

+----------+------------+----------------+
|Country_Id|Country_Name|           Coach|
+----------+------------+----------------+
|         1|   Argentina|    Damon Pruitt|
|         2|     Belgium|    Lucas Hodges|
|         3|      Brasil|   Cody Delacruz|
|         4|     England| Tobias Melendez|
|         5|     Germany|  Barry Reynolds|
|         6|       Italy|    Elmo Patrick|
|         7|      Mexico|  Kuame Gallegos|
|         8| Netherlands|      Will Smith|
|         9|    Portugal|   William Price|
|        10|       Spain|Prescott Sweeney|
+----------+------------+----------------+

None


In [31]:
extracted_match_table = hockey_src2_dataframe\
    .select("Match_Name")\
    .distinct()\

match_type_dimension_table = extracted_match_table\
    .withColumn("Match_Type_Id", rank().over(Window.orderBy(col("Match_Name"))))\
    .select("Match_Type_Id", "Match_Name")

print(match_type_dimension_table.show())

match_pandas_df = match_type_dimension_table.toPandas()
match_pandas_df.to_csv('tables/dimension/match_dimension.csv', index= False)


+-------------+-------------+
|Match_Type_Id|   Match_Name|
+-------------+-------------+
|            1|         Club|
|            2|International|
+-------------+-------------+

None


In [33]:
extracted_player_table = hockey_src1_dataframe\
    .select("Player Name", "Club Name", "country Name", "Position", "Jersey_No"
            ,col("`D. O. B.`"),"Nationality")\
    .distinct()\

extracted_player_table = extracted_player_table\
    .withColumn("Player_Id", rank().over(Window.orderBy(col("Player Name"))))\
    .withColumn("Player_Name", col("Player Name"))\
    .withColumn("Club_Name", col("Club Name"))\
    .withColumn("Country_Name", col("country Name")) ## Renaming Columns

player_dimension_table = extracted_player_table\
    .join(club_dimension_table,
          on = extracted_player_table.Club_Name == club_dimension_table.Club_Name,
          how = 'left')\
    .join(country_dimension_table,
          on = extracted_player_table.Country_Name == country_dimension_table.Country_Name,
          how = 'left')\
    .select("Player_Id", "Player_Name", "Club_Id", "Country_Id", "Position", "Jersey_No", "`D. O. B.`", "Nationality")\
    .orderBy("Player_Id")    


player_dimension_table.show()

player_pandas_df = player_dimension_table.toPandas()
player_pandas_df.to_csv('tables/dimension/player_dimension.csv', index= False)

+---------+--------------------+-------+----------+--------+---------+---------+-----------+
|Player_Id|         Player_Name|Club_Id|Country_Id|Position|Jersey_No| D. O. B.|Nationality|
+---------+--------------------+-------+----------+--------+---------+---------+-----------+
|        1|              Adrian|     21|      NULL|      LW|       88|30-Mar-85|Netherlands|
|        1|              Adrian|     22|      NULL|      LW|       88|30-Mar-85|Netherlands|
|        1|              Adrian|   NULL|         8|      LW|       88|30-Mar-85|Netherlands|
|        4|              Aguero|     24|      NULL|       F|       51|14-May-84|      Italy|
|        4|              Aguero|   NULL|         6|       F|       51|14-May-84|      Italy|
|        4|              Aguero|     29|      NULL|       F|       51|14-May-84|      Italy|
|        7|           Alberto M|     10|      NULL|      LB|       74|26-Jun-83|     Brasil|
|        7|           Alberto M|   NULL|         3|      LB|       74|

In [34]:
extracted_time_table = date_csv_dataframe\
    .select("fb_year", "quarter", "fb_month", "month_name")\
    .distinct()

time_dimension_table = extracted_time_table\
        .withColumn("Time_Id", rank().over(Window.orderBy("fb_year", "quarter", "fb_month")))\
    .withColumn("Year" , col("fb_year"))\
    .withColumn("Quarter" , col("quarter"))\
    .withColumn("Month", col("month_name"))\
    .withColumn("Month_Num", col("fb_month"))\
    .select("Time_Id", "Year", "Quarter", "Month", "Month_Num")

time_pandas_df = time_dimension_table.toPandas()
time_pandas_df.to_csv('tables/dimension/time_dimension.csv', index= False)
time_dimension_table.show()
    

+-------+----+-------+---------+---------+
|Time_Id|Year|Quarter|    Month|Month_Num|
+-------+----+-------+---------+---------+
|      1|2008|      1|  January|        1|
|      2|2008|      1| February|        2|
|      3|2008|      1|    March|        3|
|      4|2008|      2|    April|        4|
|      5|2008|      2|      May|        5|
|      6|2008|      2|     June|        6|
|      7|2008|      3|     July|        7|
|      8|2008|      3|   August|        8|
|      9|2008|      3|September|        9|
|     10|2008|      4|  October|       10|
|     11|2008|      4| November|       11|
|     12|2008|      4| December|       12|
|     13|2009|      1|  January|        1|
|     14|2009|      1| February|        2|
|     15|2009|      1|    March|        3|
|     16|2009|      2|    April|        4|
|     17|2009|      2|      May|        5|
|     18|2009|      2|     June|        6|
|     19|2009|      3|     July|        7|
|     20|2009|      3|   August|        8|
+-------+--

In [10]:
match_type_df = match_type_dimension_table.alias("match")
league_df = league_dimension_table.alias("league")
club_df = club_dimension_table.alias("club")
country_df = country_dimension_table.alias("country")
time_df = time_dimension_table.alias("time")
player_dim_df = player_dimension_table.alias("PlayerDim")

player_df = player_dimension_table\
    .select("Player_Id", "Player_Name")\
    .distinct()\
    .alias("player")

In [11]:
extracted_hockey_player_table = hockey_src1_dataframe\
    .join(time_dimension_table, 
          on = (hockey_src1_dataframe.Month == time_dimension_table.Month) & (hockey_src1_dataframe.Year == time_dimension_table.Year),
          how = 'inner')\
    .join(match_type_dimension_table,
          on = hockey_src1_dataframe.Match == match_type_dimension_table.Match_Name,
          how = 'inner')\
    .join(player_df,
          on = hockey_src1_dataframe['Player Name'] == player_df.Player_Name,
          how = 'inner')\
    .join(club_df,
          on= hockey_src1_dataframe['Club Name'] == club_df.Club_Name,
          how = 'left')\
    .join(country_df,
          on = hockey_src1_dataframe['country Name'] == country_df.Country_Name,
          how = 'left')\
    .select("Time_Id", "Match_Type_Id", "Player_Id", "Club_Id", "Country_Id", "appearances",
            "goals scored", "goals assist", "total shots", "shots on target", "fouls made",
            "fouls suffered", "yellow card", "red card", "goals saved", "goals conceded(stopped)", "total penalty", "successful penalty", "salary")\
    .orderBy("Player_Id")


player_stats_fact_table = extracted_hockey_player_table\
    .withColumn("fact_id", monotonically_increasing_id()+1 )\
    .withColumnRenamed("goals scored", "goals_scored")\
    .withColumnRenamed("goals assist", "goals_assist")\
    .withColumnRenamed("total shots", "total_shots")\
    .withColumnRenamed("shots on target", "shots_on_target")\
    .withColumnRenamed("fouls made", "fouls_made")\
    .withColumnRenamed("fouls suffered", "fouls_suffered")\
    .withColumnRenamed("yellow card", "yellow_card")\
    .withColumnRenamed("red card", "red_card")\
    .withColumnRenamed("goals saved", "goals_saved")\
    .withColumnRenamed("goals conceded(stopped)", "goals_conceded")\
    .withColumnRenamed("total penalty", "total_penalty")\
    .withColumnRenamed("successful penalty", "successful_penalty")\
    .select("fact_id","Time_Id", "Match_Type_Id", "Player_Id", "Club_Id", "Country_Id", "appearances",
            "goals_scored", "goals_assist", "total_shots", "shots_on_target", "fouls_made",
            "fouls_suffered", "yellow_card", "red_card", "goals_saved", "goals_conceded",
            "total_penalty", "successful_penalty", "salary")

player_stats_pandas_df = player_stats_fact_table.toPandas()
player_stats_pandas_df.to_csv('tables/fact/player_stats_facts_table.csv', index= False)


In [12]:
## Add month hockey_src2_dataframe
modified_hockey_team_dataframe = hockey_src2_dataframe\
    .withColumn("Month" , row_number()\
                .over(Window\
                      .partitionBy(
                                    "Year", "Match_Name", "League_Name", "Club_Name", "Coach",
                                    "Country Name", "Manager", "Owner")\
                      .orderBy(
                                    "Year", "Match_Name", "League_Name", "Club_Name", "Coach",
                                    "Country Name", "Manager", "Owner")))\
    .orderBy('Club_Name', 'Year', 'Month')

modified_hockey_team_dataframe.show()

+----+----------+-----------+------------------+-----+------------+-----------+----------+-----------+----+-----+-----+------------+---------+-----+
|Year|Match_Name|League_Name|         Club_Name|Coach|Country Name|    Manager|     Owner|appearances|wins|losts|drawn|clean sheets|Net Worth|Month|
+----+----------+-----------+------------------+-----+------------+-----------+----------+-----------+----+-----+-----+------------+---------+-----+
|2010|      Club|    LA Liga| Atheletico Madrid|   NA|          NA|Alvin Lucas|Dean Green|          8|   7|    0|    1|           5|$745,634 |    1|
|2010|      Club|    LA Liga| Atheletico Madrid|   NA|          NA|Alvin Lucas|Dean Green|          8|   2|    1|    5|           1|$745,634 |    2|
|2010|      Club|    LA Liga| Atheletico Madrid|   NA|          NA|Alvin Lucas|Dean Green|          8|   6|    1|    1|           3|$745,634 |    3|
|2010|      Club|    LA Liga| Atheletico Madrid|   NA|          NA|Alvin Lucas|Dean Green|          8|   6

In [13]:
# Apply aliases to the DataFrames before joining
hockey_team_df = modified_hockey_team_dataframe.alias("hockey_team")

# Perform the joins using the aliases
extracted_hockey_team_table = hockey_team_df\
    .join(time_df,
          (hockey_team_df.Year == time_df.Year) & (hockey_team_df.Month == time_df.Month_Num),
          how = 'inner')\
    .join(match_type_df,
          hockey_team_df.Match_Name == match_type_df.Match_Name,
          how='inner')\
    .join(league_df,
          hockey_team_df.League_Name == league_df.League_Name,
          how='inner')\
    .join(club_df,
          hockey_team_df.Club_Name == club_df.Club_Name,
          how='left')\
    .join(country_df,
          hockey_team_df['Country Name'] == country_df.Country_Name,
          how='left')\
    .withColumn("fact_id", monotonically_increasing_id() + 1)\
    .select("fact_id",
            time_df["Time_Id"],
            match_type_df["Match_Type_Id"],
            league_df["League_Id"],
            club_df["Club_Id"],
            country_df["Country_Id"],
            "appearances", "wins", "losts", "drawn", "clean sheets", "Net Worth")

team_stats_fact_table = extracted_hockey_team_table\
    .withColumnRenamed("clean sheets", "clean_sheets")\
    .withColumnRenamed("Net Worth", "net_worth")\
    .select("fact_id", "Time_Id", "Match_Type_Id", "League_Id", "Club_Id", "Country_Id",
            "appearances", "wins", "losts", "drawn", "clean_sheets", "net_worth")

# Show the resulting DataFrame
team_stats_fact_table.show()


team_stats_pandas_df = team_stats_fact_table.toPandas()
team_stats_pandas_df.to_csv('tables/fact/team_stats_facts_table.csv', index= False)


+-------+-------+-------------+---------+-------+----------+-----------+----+-----+-----+------------+---------+
|fact_id|Time_Id|Match_Type_Id|League_Id|Club_Id|Country_Id|appearances|wins|losts|drawn|clean_sheets|net_worth|
+-------+-------+-------------+---------+-------+----------+-----------+----+-----+-----+------------+---------+
|      1|     25|            1|        1|      6|      NULL|          8|   1|    3|    4|           1|$567,457 |
|      2|     26|            1|        1|      6|      NULL|          8|   4|    2|    2|           1|$567,457 |
|      3|     27|            1|        1|      6|      NULL|          8|   3|    3|    2|           1|$567,457 |
|      4|     28|            1|        1|      6|      NULL|          8|   1|    5|    2|           0|$567,457 |
|      5|     29|            1|        1|      6|      NULL|          8|   1|    5|    2|           0|$567,457 |
|      6|     30|            1|        1|      6|      NULL|          8|   7|    0|    1|       

In [14]:
player_fact_df = player_stats_fact_table.alias("PlayerFacts")
player_team_df = team_stats_fact_table.alias("TeamFacts")

In [15]:
##1
# Join player statistics with country, club, and player dimensions
player_club_country_df = player_fact_df\
    .join(country_df, # Joining with the country dimension
          on=player_fact_df.Country_Id == country_df.Country_Id,
          how = 'left')\
    .join(club_df, # Joining with the club dimension
          on = player_fact_df.Club_Id == club_df.Club_Id,
          how = 'left')\
    .join(player_df, # Joining with the player dimension
          on = player_fact_df.Player_Id == player_df.Player_Id,
          how = 'inner')\
    .select( # Selecting relevant columns
            'PlayerFacts.Player_Id',
            'player.Player_Name',
            'country.Country_Id',
            'country.Country_Name',
            'club.Club_Id',
            'club.Club_Name'
    )\
    .distinct()\
    .orderBy('PlayerFacts.Player_Id')
    # Removing duplicate entries, Ordering the result by Player_Id
    
# Display the resulting DataFrame
player_club_country_df.show()

player_club_country_pandas = player_club_country_df.toPandas()
player_club_country_pandas.to_csv('output/answer_q1.csv', index = False)

+---------+--------------------+----------+------------+-------+-------------+
|Player_Id|         Player_Name|Country_Id|Country_Name|Club_Id|    Club_Name|
+---------+--------------------+----------+------------+-------+-------------+
|        1|              Adrian|      NULL|        NULL|     21|  Real Madrid|
|        1|              Adrian|      NULL|        NULL|     22|Real Sociedad|
|        1|              Adrian|         8| Netherlands|   NULL|         NULL|
|        4|              Aguero|      NULL|        NULL|     24|   Sevilla FC|
|        4|              Aguero|      NULL|        NULL|     29|     Valencia|
|        4|              Aguero|         6|       Italy|   NULL|         NULL|
|        7|           Alberto M|      NULL|        NULL|     23|      Schalke|
|        7|           Alberto M|      NULL|        NULL|     10|      Everton|
|        7|           Alberto M|         3|      Brasil|   NULL|         NULL|
|       10|Alessandro  Florenzi|      NULL|        N

In [16]:
##2
# Joining player statistics with player dimension and match type dimension
player_stats_join_player_dimension = player_fact_df \
    .join(player_df,  # Joining with the player dimension
          on=player_fact_df.Player_Id == player_df.Player_Id,
          how='outer') \
    .join(match_type_df,  # Joining with the match type dimension
          on=player_fact_df.Match_Type_Id == match_type_df.Match_Type_Id,
          how='inner') \
    .filter(col('Match.Match_Name') == 'International') \
    .select("player.Player_Id",  # Selecting relevant columns
            "player.Player_Name",
            player_fact_df['appearances'])

# Grouping by player ID and name, and aggregating total appearances
player_appearances_dataframe = player_stats_join_player_dimension \
    .groupBy("Player_Id", "Player_Name") \
    .agg(sum("appearances").alias("total_appearances")) \
    .orderBy("total_appearances", ascending=False)  
    # Ordering by total appearances in descending order

# Ranking players by total appearances and filtering for the top player
max_player_appearances = player_appearances_dataframe \
    .withColumn("appearance_rank",  # Adding a ranking column based on total appearances
                 rank().over(Window.orderBy("total_appearances"))) \
    .select("Player_Id", "Player_Name", "total_appearances") \
    .filter(col("appearance_rank") == 1)  # Filtering to get only the top-ranked player

# Displaying results of all player appearances
player_appearances_dataframe.show()
player_appearances_dataframe_pandas = player_appearances_dataframe.toPandas()
player_appearances_dataframe_pandas.to_csv('output/answer_q2.csv', index = False)


+---------+--------------+-----------------+
|Player_Id|   Player_Name|total_appearances|
+---------+--------------+-----------------+
|      425|      Tim Krul|              288|
|      435|    Toni Kroos|              288|
|      170|         Giggs|              288|
|      186| Iker Casillas|              288|
|       49|        Buffon|              288|
|      146|     Francesco|              288|
|       58|       Carrick|              288|
|      226|   Juan Manuel|              288|
|      115|       Etienne|              288|
|      220|  Jose Enrique|              288|
|      269|    Marco Rojo|              288|
|      464|      Vladimir|              288|
|      381|    Ronaldinho|              288|
|      254|     Luke Shaw|              288|
|      173|     Hernandez|              288|
|      211| Joel Campbell|              288|
|       64| Cesc Fabregas|              288|
|      352|Radamel Falcao|              288|
|      223|    Juan Isaac|              288|
|      183

In [17]:
##3
# Grouping and Aggregation
goals_df = player_fact_df\
    .groupBy('Player_Id')\
    .agg(sum('goals_scored').alias('goals_scored'))

goals_df = goals_df\
    .join(player_df, # Joining with the player dimension
          on = player_df.Player_Id == goals_df.Player_Id,
          how = 'inner')\
    .select('player.Player_Id','player.Player_Name', 'goals_scored')\
    .distinct()\
    .orderBy('goals_scored', ascending = False)
    # Selecting distinct, Ordering by goals scored descending
    
max_goals_df = goals_df\
    .withColumn('goal_rank', rank().over(Window.orderBy(col('goals_scored').desc())))\
    .select('Player_Id', 'Player_Name', 'goals_scored')\
    .filter(col('goal_rank') <= 10)

max_goals_pandas_df = max_goals_df.toPandas()
max_goals_pandas_df.to_csv('output/answer_q3.csv', index = False)


In [24]:
##4

# Grouping and Aggregation
red_card_df = player_fact_df\
    .groupBy('Player_Id')\
    .agg(sum('red_card').alias('red_cards'))

red_card_df = red_card_df\
    .join(player_dim_df, # Joining with the player dimension
          on = player_dim_df.Player_Id == red_card_df.Player_Id,
          how = 'inner')\
    .select('PlayerDim.Player_Id','PlayerDim.Player_Name', 'red_cards')\
    .distinct()\
    .orderBy('red_cards', ascending = False)
    # Selecting distinct, Ordering by red cards descending 
    
max_red_cards_df = red_card_df\
    .withColumn('card_rank', rank().over(Window.orderBy(col('red_cards').desc())))\
    .select('Player_Id', 'Player_Name', 'red_cards')\
    .filter(col('card_rank') <= 5)

max_red_cards_pandas = max_red_cards_df.toPandas()
max_red_cards_pandas.to_csv('output/answer_q4.csv', index = False)


In [25]:
## 5
club_wins_df = player_team_df\
    .groupBy('Club_Id')\
    .agg(sum('wins').alias('wins'))  

club_wins_df = club_wins_df\
    .join(club_df,          
          on=club_df.Club_Id == club_df.Club_Id,          
          how='inner')\
    .filter(col('Club_Name') != 'NA')\
    .select(club_wins_df["Club_Id"],"Club_Name", "wins")\
    .orderBy("wins", ascending = False)

max_club_wins_df = club_wins_df\
    .withColumn('win_rank', rank().over(Window.orderBy(col('wins').desc())))\
    .select('Club_Id', 'Club_Name', 'wins')\
    .filter(col('win_rank') <= 5)

max_club_wins_pandas = max_club_wins_df.toPandas()
max_club_wins_pandas.to_csv('output/answer_q5.csv', index = False)